# 0. Imports

In [1]:
import kenlm
import re
import glob

import pandas as pd
import numpy as np
 
#PATH = r"C:\Users\barka\Desktop\NLP\Data\"
PATH = r"C:\Users\Louis\Documents\University\Masters\A23\NLP\Devoirs\data\hw1\train"

# Question 1
Nous avons lancé cette commande pour entraîner le modèle:

```
code!
```

# Question 2

# a. Model Stats

### Load model

In [22]:
model_kenlm = kenlm.LanguageModel(r"C:\Users\Louis\Documents\University\Masters\A23\NLP\model2G.arpa")
#model_lmppl = lmppl.LM("C:\\Users\\Louis\\Documents\\University\\Masters\\A23\\NLP\\model2G.arpa")

### Calculate perplexity

In [23]:
f = open(PATH, "r", encoding="utf-8")

sentences = f.readlines()[:1000]

perplexities = []
for sentence in sentences:
    sentence_filtered = sentence.replace('\n', '')
    score = model_kenlm.perplexity(sentence_filtered)
    perplexities.append(score)

### Statistics

In [24]:
perp_df = pd.Series(perplexities)

perp_df.describe()

count     1000.000000
mean       412.933038
std       1026.208320
min         28.735342
25%        145.652765
50%        227.562864
75%        419.894973
max      21411.129543
dtype: float64

# b. Slices Performance

# c. Best Model

# d. Small Questions

### i - Lines

In [19]:
# Concatenate all files
all_slices = pd.Series()
for slice_path in glob.glob(PATH+"\*.txt"):
    slice_df = pd.read_table(slice_path, header=None)
    slice_df = slice_df[0] # Make it a Series

    all_slices = pd.concat([all_slices, slice_df])
all_slices = all_slices.reset_index(drop=True)

all_slices

C:\Users\Louis\AppData\Local\Temp\ipykernel_4068\506740737.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_slices = pd.Series()


0          The U.S. Centers for Disease Control and Preve...
1          When Ms. Winfrey invited Suzanne Somers to sha...
2          Elk calling -- a skill that hunters perfected ...
3                                                   Don 't !
4          Fish , ranked 98th in the world , fired 22 ace...
                                 ...                        
2471015     I think the reason Darren Aronofsky wanted to...
2471016    We 've asked the police to help me trace where...
2471017    Wild spending and family dysfunction are commo...
2471018     Many customers keep asking about ' Rambo 4 ' ...
2471019    Where Ubisoft and developer Gameloft really do...
Length: 2471020, dtype: object

### ii - Repeats

In [39]:
# Count repeats
sentence_counts = all_slices.value_counts()
seen_3 = sentence_counts[sentence_counts>=3]
seen_4 = sentence_counts[sentence_counts>=4]

In [40]:
seen_3

American negotiator Watson said the Bush administration is planning probably four more meetings in the Major Economies series before a " leaders ' meeting " in mid-2008 presents a final outcome .    3
dtype: int64

In [41]:
seen_4

Series([], dtype: int64)

### iii - Characters

In [25]:
sentence_lengths = all_slices.str.len()
sentence_lengths_counts = sentence_lengths.value_counts()
length_20 = len(sentence_lengths_counts[sentence_lengths_counts==20])
length_120 = len(sentence_lengths_counts[sentence_lengths_counts==120])

print(f"Number of occurences per caracter lengths: \n20: {length_20}\n120: {length_120}")

Number of occurences per caracter lengths: 
20: 35
120: 1


### iv - Max & min

In [37]:
print(f"Shortest sentence length: {sentence_lengths.min()}\nLongest sentence length: {sentence_lengths.max()}")

Shortest sentence length: 2
Longest sentence length: 9572


### v - Short sentence frequency

In [44]:
less_21 = all_slices[sentence_lengths < 21]
less_21_counts = less_21.value_counts()
less_21_counts.head(100).to_csv(PATH+"\\frequent_short_sentences.csv")

### vi - Word frequency

In [8]:
# Concatenate everything into a single string
all_slices_str = ""

pattern = r'[^\w\s]|[\d]' # Regex to select punctuation and digits
for slice_path in glob.glob(PATH+"\*.txt"):
    f = open(slice_path, "r", encoding="utf-8")
    slice_str = f.read()

    # Replacing punctuation, digits and skip lines
    slice_str = re.sub("\n", ' ', slice_str) # Skipline becomes space
    slice_str = re.sub(pattern, '', slice_str).lower()  # Remove punctuation and digits, then put it in lowercase
    
    # Concatenating
    all_slices_str = all_slices_str + slice_str

# Split over the spaces, removing them and putting them into pandas
words = all_slices_str.split(" ")
word_df = pd.Series(words)

word_counts = word_df.value_counts()
word_counts = word_counts.drop(["", "continue", "to"]) # Drop continue, to, and '' which caused me problems
word_counts.head(6)

the    3751181
of     1584635
a      1501086
and    1471456
in     1347002
s       650276
dtype: int64

### vii - First 10  words

In [6]:
sentence_starts = []

pattern = r'[^\w\s]|[\d]' # Regex to select punctuation and digits
for slice_path in glob.glob(PATH+"\*.txt"):
    f = open(slice_path, "r", encoding="utf-8")
    slice_str = f.readlines()

    for sentence in slice_str:
        sentence = sentence.split(" ")
        first_10 = sentence[:10]
        
        first_10_str = ""
        for word in first_10:
            first_10_str = first_10_str + " " + word
        sentence_starts.append(first_10_str)

sentence_starts = pd.Series(sentence_starts)
sentence_starts.value_counts()

 A version of this article appeared in print on March                   113
 The yield on the benchmark 10-year Treasury note , which               109
 A version of this article appeared in print on January                  77
 Enterprise computing , economics of technology , Microsoft , I.B.M.     69
 This entry was posted by Gwen Robinson on Wednesday ,                   64
                                                                       ... 
 Even though his land is only six miles from official                     1
 The policy was designed to control the country 's exploding              1
 Rogge urged China to respect its " moral engagement "                    1
 They also include zebra and quagga mussels , which have                  1
 Where Ubisoft and developer Gameloft really do pull through is           1
Length: 2738435, dtype: int64

### viii - Maj

In [28]:
# Concatenate everything into a single string
all_slices_str = ""

pattern = r'[^\w\s]|[\d]' # Regex to select punctuation and digits
for slice_path in glob.glob(PATH+"\*.txt"):
    f = open(slice_path, "r", encoding="utf-8")
    slice_str = f.read()

    # Replacing punctuation, digits and skip lines
    slice_str = re.sub("\n", ' ', slice_str) # Skipline becomes space
    slice_str = re.sub(pattern, '', slice_str)  # Remove punctuation and digits
        
    # Concatenating
    all_slices_str = all_slices_str + slice_str

# Split over the spaces, removing them and putting them into pandas
words = all_slices_str.split(" ")
word_df = pd.Series(words)

word_df_upper = word_df[word_df.str.isupper()]
word_df_upper_4 = word_df_upper[word_df_upper.str.len()==4]
word_df_upper_4_counts = word_df_upper_4.value_counts()
word_df_upper_4_counts.to_csv(PATH + r"\most_common_4maj_words.csv")
word_df_upper_4_counts.head(5)

YORK    5512
NATO    3133
NASA    2238
NYSE    1720
AIDS    1437
dtype: int64